# SAP Standard Example

(tweaked for MTA Projects  & HDI Containers)


In [3]:
## Import and Setup for using HDI Schema
from hana_ml import dataframe
from hana_ml.algorithms import classification
dbSchema = 'HANA_ML_DEMO_HDI_DB_1'
host =   'hxehost'
port =   '39015'
user =   'MLUSER'
password = 'Password1'

#define DataFrames
connection_context = dataframe.ConnectionContext(host, int(port), user, password)
df = connection_context.table('examples.DATA_TBL_FIT', schema=dbSchema)  #'PAL_LOGISTICR_DATA_TBL'

#Train
lr = classification.LogisticRegression(connection_context, solver='newton',
                                        thread_ratio=0.1, max_iter=1000,
                                        categorical_variable=['V3'], pmml_export=1,
                                        stat_inf=True, tol=0.000001)
lr.fit(df, ['V1', 'V2', 'V3'], 'CATEGORY', False)

#Predict
pred_df = connection_context.table('examples.DATA_TBL_PREDICT', schema=dbSchema)  #'PAL_FLOGISTICR_PREDICTDATA_TBL'
result = lr.predict(pred_df, ['V1', 'V2', 'V3'])
result.collect()

,ID,CLASS,SCORE,COMPLEMENT_SCORE,LOG_SCORE,LOG_COMPLEMENT_SCORE,DECISION_FUNC
0,0,1,9.503656e-01,0.049634,-0.050909,-3.003071e+00,-2.952163
1,1,1,8.485314e-01,0.151469,-0.164248,-1.887377e+00,-1.723129
2,2,1,9.555893e-01,0.044411,-0.045427,-3.114276e+00,-3.068849
3,3,0,3.702131e-02,0.962979,-3.296262,-3.772400e-02,3.258538
4,4,0,2.229288e-02,0.977707,-3.803488,-2.254513e-02,3.780943
5,5,0,2.504115e-01,0.749589,-1.384650,-2.882309e-01,1.096419
6,6,0,4.946187e-02,0.950538,-3.006553,-5.072701e-02,2.955826
7,7,0,9.922804e-03,0.990077,-4.612920,-9.972363e-03,4.602947
8,8,0,2.853014e-01,0.714699,-1.254209,-3.358944e-01,0.918315
9,9,0,2.689367e-01,0.731063,-1.313279,-3.132552e-01,1.000024


# SCIKIT Documentation Example

In [63]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
#print (X)  
#print (y)
clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000,
                          multi_class='multinomial').fit(X, y)
#print (X[:2, :])
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :]) 
clf.score(X, y)

0.9733333333333334

# SCIKIT Example with Hana Dataset

In [100]:
## Import and Setup for using HDI Schema
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from hana_ml import dataframe

labelencoder = LabelEncoder()

dbSchema = 'HANA_ML_DEMO_HDI_DB_1'
host =   'hxehost'
port =   '39015'
user =   'MLUSER'
password = 'Password1'


#define DataFrames
connection_context = dataframe.ConnectionContext(host, int(port), user, password)
df = connection_context.table('examples.DATA_TBL_FIT', schema=dbSchema)  #'PAL_LOGISTICR_DATA_TBL'

#Prepare Features Columns as numpy ndarray and Depedant Variable Column as 
dfX = df[['V1', 'V2', 'V3']].collect().values
#print (dfX)
dfy = df[['CATEGORY']].collect().values.ravel()
#print (dfy)

# Encoding V1 ALPHA 
dfX[:, 0] = labelencoder.fit_transform(dfX[:, 0])
#print (dfX[:5,:])

#train
clf = LogisticRegression(random_state=0, solver='newton-cg', max_iter=1000,
                          multi_class='multinomial').fit(dfX, dfy)


### Prepare Prediction dataset
pred_df = connection_context.table('examples.DATA_TBL_PREDICT', schema=dbSchema)  #'PAL_FLOGISTICR_PREDICTDATA_TBL'
pred_dfX = pred_df[['V1', 'V2', 'V3']].collect().values
# Encoding V1 ALPHA 
pred_dfXoriginal =  pred_dfX.copy()
pred_dfX[:, 0] = labelencoder.fit_transform(pred_dfX[:, 0])
#print (pred_dfX[:5,:])


##Predict
predictedCategory      =  clf.predict(pred_dfX)
predictedCategoryProba =  clf.predict_proba(pred_dfX) 

##Show output
resultDF         = pd.DataFrame(pred_dfXoriginal)
resultDF.columns = resultDF.columns+1
resultDF.index   = resultDF.index + 1
resultDF         = resultDF.rename(columns = {1:'V1', 2:'V2', 3:'V3'})
resultDF['predictedCATEGORY'] = predictedCategory
resultDF['predictedProbaCAT0'] = predictedCategoryProba[:,0]
resultDF['predictedProbaCAT1'] = predictedCategoryProba[:,1]

resultDF 

#clf.score(dfX, dfy)

,V1,V2,V3,predictedCATEGORY,predictedProbaCAT0,predictedProbaCAT1
1,B,2.62,0,1,0.206817,0.793183
2,B,2.875,0,1,0.329367,0.670633
3,A,2.32,1,1,0.142869,0.857131
4,A,3.215,2,0,0.720803,0.279197
5,B,3.44,3,0,0.904241,0.095759
6,B,3.46,0,0,0.677329,0.322671
7,A,3.57,1,0,0.787868,0.212132
8,B,3.19,2,0,0.751515,0.248485
9,A,3.15,3,0,0.786651,0.213349
10,B,3.44,0,0,0.666382,0.333618
